In [ ]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
response = sg.client.contactdb.recipients.count.get()
print(response.status_code)
print(response.body)
print(response.headers)

UnauthorizedError: HTTP Error 401: Unauthorized

parse index.xml to get latest feed item

In [10]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(feed.feed.title + ": " + newsletter.title)
print(newsletter.description)

newsletter: Post With Code
<p>This is a post with executable code.</p>


In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
from_email = Email("brad.stenger@gmail.com")
to_email = To("jstenger@uvm.edu")
subject = feed.feed.title + ": " + newsletter.title
content = Content(MimeType.html, newsletter.description)
mail = Mail(from_email, to_email, subject, content)
response = sg.client.mail.send.post(request_body=mail.get())
print(response.status_code)
print(response.body)
print(response.headers)

UnauthorizedError: HTTP Error 401: Unauthorized